## Greedy Algorithm

In [1]:
import os, sys
sys.path.append("..")
sys.path.append("..\\..")

In [2]:
from opttrot.pauli import PauliElement
from opttrot.pauli_utils import sym_code2pstr

import numpy as np

In [3]:
N = 3

In [4]:
p1 = PauliElement(2, 2, N, 1)

In [5]:
parr = np.array([PauliElement(0, 0, N, 1), PauliElement(1, 0, N, 1), PauliElement(0, 1, N, 1)])
parr

array([PauliElement(n=3, weight=1.000000+(0.000000)j, III),
       PauliElement(n=3, weight=1.000000+(0.000000)j, IIX),
       PauliElement(n=3, weight=1.000000+(0.000000)j, IIZ)], dtype=object)

In [6]:
parr[0] = p1@parr[0]

In [7]:
parr[2] = p1@parr[2]

In [8]:
parr

array([PauliElement(n=3, weight=1.000000+(0.000000)j, IYI),
       PauliElement(n=3, weight=1.000000+(0.000000)j, IIX),
       PauliElement(n=3, weight=1.000000+(0.000000)j, IYZ)], dtype=object)

In [9]:
int(True)

1

In [10]:
tuple1 = (1,3)
tuple2 = (5,2)
tuple3 = (1,4)
tuple(sorted(tuple(sorted(t)) for t in [tuple1, tuple2, tuple3]))

((1, 3), (1, 4), (2, 5))

In [11]:
from copy import copy
class PauliFrame:
    def __init__(self, n):
        self.qubit = n

        self.front = np.vstack([np.zeros(self.qubit), 2**np.arange(self.qubit)]).T.astype(int)
        self.back  = np.vstack([2**np.arange(self.qubit), np.zeros(self.qubit)]).T.astype(int)
        self.sign = np.ones(self.qubit, dtype=int)

        self.history = []
    def __repr__(self):
        reps = []
        for i, (x1, z1, x2, z2)  in enumerate(np.hstack([self.front, self.back])):
            reps.append(f"{self.sign[i]}, {sym_code2pstr((x1, z1),self.qubit)}, {sym_code2pstr((x2, z2), self.qubit)}\n")
        return "".join(reps)
    def _tuple_sort_triple(self, ts):
        return tuple(sorted(tuple(sorted(t)) for t in ts))
    def __eq__(self, other):
        self_set = set()
        other_set = set()
        for si, sj in zip(self.front, self.back):
            sk, _ = self._product(si, sj)
            self_set.add(self._tuple_sort_triple([si, sj, sk]))
        for si, sj in zip(other.front, other.back):
            sk, _ = self._product(si, sj)
            other_set.add(self._tuple_sort_triple([si, sj, sk]))
        return self_set <= other_set
    def __ne__(self, other):
        return not self.__eq__(other)
    def __hash__(self):
        self_list = list()
        for si, sj in zip(self.front, self.back):
            sk, _ = self._product(si, sj)
            self_list.append(self._tuple_sort_triple([si, sj, sk]))
        sorted(self_list)
        return hash(tuple(self_list))
    @property
    def matrix(self):
        return np.hstack([self.front, self.back])
    
    def relative_supp(self, p):
        supp = 0
        for (si, si_t) in zip(self.front, self.back):
            supp += int(self._commute(si, p)|self._commute(si_t, p))
        return supp

    def copy(self):
        pf  = PauliFrame(self.qubit)
        pf.front = copy(self.front)
        pf.back = copy(self.back)
        pf.sign = copy(self.sign)
        pf.history = copy(self.history)
        return pf
    def get_previous(self):
        pf = self.copy()
        (i, j, c_type, hash_key) = pf.history[-1]
        pf.two_entangle(i, j, c_type)
        del(pf.history[-1])
        del(pf.history[-1])
        return pf
    def h(self, i):
        self.back[i][0], self.front[i][0] = self.front[i][0], self.back[i][0]
        self.back[i][1], self.front[i][1] = self.front[i][1], self.back[i][1]
    def s(self, i):
        si, si_t = (self.front[i][0], self.front[i][1]), (self.back[i][0], self.back[i][1])
        new_si_t, _ = self._product(si, si_t)
        self.back[i][0] = new_si_t[0]
        self.back[i][1] = new_si_t[1]
        pass
    def s_d(self, i):
        self.s(i)
        self.sign[i] *= -1  
        pass 
    def _commute(self, p1, p2):
        nx1, nz1 =  p1
        nx2, nz2 =  p2
        a = bin(nx1&nz2).count("1")%2
        b = bin(nx2&nz1).count("1")%2
        return a==b

    def _product(self, p1, p2):
        nx1, nz1 = p1
        nx2, nz2 = p2

        # update sign 
        #f=0
        #tmp1 = nx1 & nz2
        #tmp2 = nx2 & nz1
        #f += bin(tmp1).count("1")
        #f += bin(tmp2).count("1")
        #f += 2*bin(nx1&nz2).count("1")

        return ((nx1 ^ nx2, nz1 ^ nz2), 1)
    
    def  _basis(self, i, j, c_type, front=True):
        if c_type ==0:
            return None
        if c_type ==1:
            #cz
            self.h(j)
            return None
        if c_type == 2:
            # cy
            if front:
                self.s_d(j)
            else:
                self.s(j)
            return None
        if c_type ==3:
            self.h(i)
            return None
        if c_type ==4:
            self.h(i)
            if front:
                self.s_d(j)
            else:
                self.s(j)
            return None
        if c_type==5:
            if front:
                self.s_d(i)
                self.h(i)
                self.s_d(j)
            else:
                self.s(i)
                self.h(i)
                self.s(j)
            return None


    def two_entangle(self, i, j, c_type = 0):
        assert i>=0 and j>=0, "i, j must be positive integer."
        assert i<self.qubit and j<self.qubit, "Exceeding the maximum index."

        self._basis(i, j , c_type, front=True)
        
        # CX
        si, si_t = (self.front[i][0], self.front[i][1]), (self.back[i][0], self.back[i][1])
        sj, sj_t = (self.front[j][0], self.front[j][1]), (self.back[j][0], self.back[j][1])

        new_sj, _ = self._product(sj, si)
        new_si_t, _ = self._product(si_t, sj_t)

        self.front[j][0] = new_sj[0]
        self.front[j][1] = new_sj[1]
        self.back[i][0] = new_si_t[0]
        self.back[i][1] = new_si_t[1]
        
        self._basis(i, j , c_type, front=False)

        self.history.append((i, j, c_type, hash(self)))

In [12]:
pf = PauliFrame(5)
pf

1, IIIIZ, IIIIX
1, IIIZI, IIIXI
1, IIZII, IIXII
1, IZIII, IXIII
1, ZIIII, XIIII

In [13]:
pf = PauliFrame(5)
pf.two_entangle(0, 4, 1)
pf.two_entangle(0, 1, 0)
pf.two_entangle(2, 4, 0)
pf

1, IIIIZ, ZIIXX
1, IIIZZ, IIIXI
1, IIZII, XIXIZ
1, IZIII, IXIII
1, ZIZII, XIIIZ

In [14]:
pf.history

[(0, 4, 1, 3061629624854387302),
 (0, 1, 0, 7102544670175337138),
 (2, 4, 0, 6865725205306480262)]

In [15]:
pf2 = PauliFrame(5)
pf2.two_entangle(0, 4, 1)
pf2.two_entangle(0, 1, 0)
pf2

1, IIIIZ, ZIIXX
1, IIIZZ, IIIXI
1, IIZII, IIXII
1, IZIII, IXIII
1, ZIIII, XIIIZ

In [16]:
pf2.history

[(0, 4, 1, 3061629624854387302), (0, 1, 0, 7102544670175337138)]

In [17]:
pf_pre = pf.get_previous()

In [18]:
pf_pre.history

[(0, 4, 1, 3061629624854387302), (0, 1, 0, 7102544670175337138)]

In [19]:
pf2 == pf_pre

True

In [20]:
pf3 = PauliFrame(5)
pf3.two_entangle(2, 4, 0)
pf3.two_entangle(0, 4, 1)
pf3.two_entangle(0, 1, 0)
pf3

1, IIIIZ, ZIZXX
1, IIIZZ, IIIXI
1, IIZII, XIXII
1, IZIII, IXIII
1, ZIZII, XIIIZ

In [21]:
pf==pf3

False

In [22]:
pf, pf3

(1, IIIIZ, ZIIXX
 1, IIIZZ, IIIXI
 1, IIZII, XIXIZ
 1, IZIII, IXIII
 1, ZIZII, XIIIZ,
 1, IIIIZ, ZIZXX
 1, IIIZZ, IIIXI
 1, IIZII, XIXII
 1, IZIII, IXIII
 1, ZIZII, XIIIZ)

In [23]:
pf4 = PauliFrame(5)
pf4.two_entangle(2, 4, 0)
pf4.two_entangle(0, 1, 0)
pf4.two_entangle(0, 4, 1)
pf4

1, IIIIZ, ZIZXX
1, IIIZZ, IIIXI
1, IIZII, XIXII
1, IZIII, IXIII
1, ZIZII, XIIIZ

In [24]:
pf5 = PauliFrame(5)
pf5.two_entangle(0, 1, 0)
pf5.two_entangle(2, 4, 0)
pf5.two_entangle(0, 4, 1)
pf5

1, IIIIZ, ZIZXX
1, IIIZZ, IIIXI
1, IIZII, XIXII
1, IZIII, IXIII
1, ZIZII, XIIIZ

In [25]:
pf6 = PauliFrame(5)
pf6.two_entangle(0, 1, 0)
pf6.two_entangle(0, 4, 1)
pf6.two_entangle(2, 4, 0)
pf6

1, IIIIZ, ZIIXX
1, IIIZZ, IIIXI
1, IIZII, XIXIZ
1, IZIII, IXIII
1, ZIZII, XIIIZ

In [26]:
pf5, pf6

(1, IIIIZ, ZIZXX
 1, IIIZZ, IIIXI
 1, IIZII, XIXII
 1, IZIII, IXIII
 1, ZIZII, XIIIZ,
 1, IIIIZ, ZIIXX
 1, IIIZZ, IIIXI
 1, IIZII, XIXIZ
 1, IZIII, IXIII
 1, ZIZII, XIIIZ)

In [27]:
pf5 == pf6

False

In [28]:
pf5.matrix

array([[ 0,  1,  3, 20],
       [ 0,  3,  2,  0],
       [ 0,  4, 20,  0],
       [ 0,  8,  8,  0],
       [ 0, 20, 16,  1]])

In [29]:
pf6.matrix

array([[ 0,  1,  3, 16],
       [ 0,  3,  2,  0],
       [ 0,  4, 20,  1],
       [ 0,  8,  8,  0],
       [ 0, 20, 16,  1]])

In [30]:
pf.matrix

array([[ 0,  1,  3, 16],
       [ 0,  3,  2,  0],
       [ 0,  4, 20,  1],
       [ 0,  8,  8,  0],
       [ 0, 20, 16,  1]])

In [31]:
pf2.matrix

array([[ 0,  1,  3, 16],
       [ 0,  3,  2,  0],
       [ 0,  4,  4,  0],
       [ 0,  8,  8,  0],
       [ 0, 16, 16,  1]])

In [32]:
pf3.matrix

array([[ 0,  1,  3, 20],
       [ 0,  3,  2,  0],
       [ 0,  4, 20,  0],
       [ 0,  8,  8,  0],
       [ 0, 20, 16,  1]])

In [33]:
from itertools import combinations, permutations

In [34]:
list(combinations([1,2,3], 2))

[(1, 2), (1, 3), (2, 3)]

In [35]:
list(combinations(range(3), 2))

[(0, 1), (0, 2), (1, 2)]

In [36]:
list(permutations(range(3), 2))

[(0, 1), (0, 2), (1, 0), (1, 2), (2, 0), (2, 1)]

## Test

In [4]:
from gen_qubits import PauliFrame

In [9]:
pf = PauliFrame(3)
print(hash(pf))
pf.two_entangle(0, 1, c_type=0)
print(hash(pf))
pf.two_entangle(1, 2, c_type=0)
print(hash(pf))
pf

7934267287378752615
1895826367948997221
7770280598866912735


1, IIZ, IXX
1, IZZ, XXI
1, ZZZ, XII

In [10]:
pf.history

[(0, 1, 0, 1895826367948997221), (1, 2, 0, 7770280598866912735)]

In [6]:
hash(pf)

7770280598866912735

In [38]:
pf = PauliFrame(3)
pf.two_entangle(1, 2, c_type=0)
pf.two_entangle(0, 1, c_type=0)
pf

1, IIZ, XXX
1, IZZ, XXI
1, ZZI, XII

## DataSet

1. Controll gate를 모든 경우에 대해 적용시켜, B frame data를 만든다.
   1. Frame 종류가 들어간 data,
   2. Frame 별로 어느 Frame에서 조작을 해서 도달했는지 B_i = B_{i+1} mapping data를 만든다.
2. H가 주어지면, Market 문제로 치환할 수 있다.
### Data set generation

- qubit 2: 12 (0.1 s)
- qubit 3: 3384 (1 min 16 s)
- qubit 4: >=4,433,615 (>= 276min)

In [3]:
from gen_qubits import run

In [5]:
run(N=3, ppols=10)

0-th:

Processing Frame 0: 100%|██████████| 1/1 [00:00<00:00,  2.79it/s]


15
1-th:

Processing Frame 1: 100%|██████████| 15/15 [00:00<00:00, 29.44it/s]


235
2-th:

...

Processing Frame 5: 100%|██████████| 3382/3382 [00:12<00:00, 281.17it/s]


3384
6-th:

Processing Frame 6: 100%|██████████| 3384/3384 [00:12<00:00, 262.38it/s]


3384


True

In [40]:
import pickle
from itertools import permutations

In [51]:
N = 6

In [52]:
pf = PauliFrame(N)
Frames = [set([pf])]
i=0
frames_len = 1

num_core = 4

In [53]:
frames = set()
frames = frames|(Frames[0])
for k in range(0, 4):
    print(f"{k}-th:", end="")
    ith_set = set()
    for p in Frames[k]:
        for i, j in permutations(range(N), 2):
            p1 = p.copy()
            p2 = p.copy()
            p3 = p.copy()
            p4 = p.copy()
            p5 = p.copy()
            p6 = p.copy()

            p1.two_entangle(i, j, c_type=0) 
            p2.two_entangle(i, j, c_type=1) 
            p3.two_entangle(i, j, c_type=2) 
            p4.two_entangle(i, j, c_type=3) 
            p5.two_entangle(i, j, c_type=4) 
            p6.two_entangle(i, j, c_type=5) 
            ith_set.add(p1)
            ith_set.add(p2)
            ith_set.add(p3)
            ith_set.add(p4)
            ith_set.add(p5)
            ith_set.add(p6)
    
    
    print(len(ith_set))
    Frames.append((ith_set))
    if frames >= ith_set:
        break
    frames = frames|Frames[k+1]
    

0-th:75
1-th:5356
2-th:

KeyboardInterrupt: 

In [ ]:
with open(f"qubit{N}_PauliFrame.data", "wb") as file:
    pickle.dump(Frames, file)
    pickle.dump(frames, file)
    

In [90]:
with open("qubit2_PauliFrame.data", "rb") as file:
    Frames = pickle.load(file)
    frames = pickle.load(file)
    

In [ ]:
# Pandas 

## Dynamic programming to find a evolution circuit.